## Domain: Industrial Safety Support - NLP based ChatBot - Safex Pybot Version 1.0 

#### Context : 

**** Great Learning Company is looking forward to design an automation which can interact with the user, understand the problem      and display the resolution procedure 

**** The company is looking for a designed chatbot utility which can help the professionals to highlight the safety risk as per      the incident description usign a ML or DL or redirect the request to an actual human support executive if the request is        complex or not in its database.

#### Data Description : 

**** The corpus is attached for the reference. Please enhance/add more data to the corpus using your linguistics skills.

In [1]:
# import the libraries

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tensorflow as tf
import tflearn
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\U6031033\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [199]:
# import our chat-bot intents file
import json
with open('GL+Bot(1).json') as json_data:
    intents = json.load(json_data)
    

In [200]:
# Initialize data fields for our file

words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

80 documents
14 classes ['employee, polyurethane, pully, installing, impact', 'employee,auxillary,hot,hitting,crane,impact', 'employee,electric,coil,warehouse,forklift,beehive,disturbed,other,people,impact', 'employee,inspect,Jumbo,equipment,esengrasante,toxicity,bottle,impact', 'employee,power,cables,disconnect,industrial,impact', 'escalation', 'greetings', 'mining,Abratech,company,putty,conditioning,tank,impact', 'mining,loosening,trucks,cylindrical,bolt,power,steering,impact', 'mining,mechanical,assistant,crane,descending,ladder,impact', 'mining,moving,oil,cylinder,rail,track,impact,wearing,leather-type,safety,gloves', 'mining,third,remote,employee,sudden,illness,impact', 'rude', 'sendoff']
200 unique stemmed words ['15', 'a', 'abratech', 'accid', 'aiml', 'am', 'an', 'and', 'any', 'anyon', 'ar', 'assist', 'at', 'auxil', 'bad', 'batch', 'bear', 'beeh', 'being', 'belong', 'best', 'bloody', 'bolt', 'bot', 'buddy', 'by', 'bye', 'cabl', 'cathod', 'chang', 'coil', 'com', 'company', 'condi

In [201]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\U6031033\AppData\Local\Temp/ipykernel_3884/1193693976.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [202]:
# reset underlying graph data
from tensorflow.python.framework import ops
ops.reset_default_graph()

# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 16)
net = tflearn.fully_connected(net, 16)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 9999  | total loss: 0.00723 | time: 0.034s
| Adam | epoch: 1000 | loss: 0.00723 - acc: 0.9997 -- iter: 72/80
Training Step: 10000  | total loss: 0.00676 | time: 0.037s
| Adam | epoch: 1000 | loss: 0.00676 - acc: 0.9998 -- iter: 80/80
--
INFO:tensorflow:C:\Users\U6031033\Desktop\Capstone Project\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [203]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [204]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('GL+Bot(1).json') as json_data:
    intents = json.load(json_data)

In [205]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\U6031033\Desktop\Capstone Project\model.tflearn


In [230]:
# functions for cleansing the sentenses and to get the bag of words

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [231]:
# functions to classify the sentences and for respective responses
# create a data structure to hold user context

context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)
            

In [232]:
classify('how are you?')


[('greetings', 0.9898449)]

In [233]:
classify("what happend to the third party remote employee?")

[('mining,third,remote,employee,sudden,illness,impact', 0.9537687)]

In [234]:
# few tests for the same

response('is anyone there?')
print()

response('is anyone there?')
print()

response('is anyone there?')
print()

response('how may i help you?')
print()

response('what incident occurred to the third party mechanical assistant working in a crane truck?')
print()

response('what the hell')
print()

response('you did not help me')
print()

response('not good solution')
print()

response('bye')
print()


Hi there! How may i help you please?

Hi there! How may i help you please?

Hello! how could i help you?

Hello! how could i help you?

After throwing the wooden blocks of support of the stabilizer of the Hiab crane truck to the ground, the mechanic assistant was descending from the truck using ladder, he jumped from the last step of the ladder from 69cm, The mechanical assistant jumped from the last step of the ladder and set his left foot on the edge of one of the wooden blocks and got injured on his left ankle, It is an accident at a level of I and its potential level is of IV

Please use respectful words

Apologies... Tarnsferring the request to our Second Level Team...

Apologies... Tarnsferring the request to our Second Level Team...

I hope I was able to assist you, Good Bye



In [235]:
# we can try some other questions which are given in the project .json file

# 1. Start chat session with greetings and ask what the user is looking for.
# 2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus.
# 3. End the chat session only if the user requests to end else ask what the user is looking for. 
# Loop continues till the user asks to end it.


response('hello')
print()
response('what is the impact to the employee by accidents of working on a heating bearing and hitting with a hammer?')
print()
response('what may happen when an employee disconnects the power cables?')
print()
response('why the employees wearing leather-type safety gloves and placing the mobile platform on the rail track?')
print()
response('screw you')
print()
response('who worked on crane truck and descending from the crane using ladder?')
print ()
response('what is the impact to any person who works for Abratech company?')
print()
response('you did not help me')
print()
response('thanks')



Hello! How do you do? How could i help you today?

When the employee heated a bearing and hit with the hammer and chisel at one end of the bearing track, a piece of bearing is detached, The employee got a cut on his right thigh and he was transferred to the hospital in an ambulance immediately, Fortunately it is an accident at a level of I and its potential level is of II

While the employee was making a disconnection of two power cables of the gate in order to remove it, When the employee was removing the rope tied to the body of gate the warning post was pulled over and it hit the helmet of other employee who was standing at his side, Fortunately it is an accident at a level of I and its potential level is of III

When the two employees are preparing to move an oil cylinder on a mobile platform of about 200ks which is mounted on rails, the mobile platform derailed from the rail tracks, both the employees wore leather-type safety gloves and tried to lift the mobile platform and keep i

In [236]:
# the same can be implemented using interactive mode assuming as a chat bot like below:


import numpy as np
from keras.preprocessing.sequence import pad_sequences


print("##########################################")
print("#       Start Chatting ver. 1.0          #")
print("##########################################")


stmt = ""
while stmt != 'q':
    stmt  = input("you : ")
    if stmt == 'quit':
        break
    else:
        response(stmt)
        print()
 


##########################################
#       Start Chatting ver. 1.0          #
##########################################
you : hi
Hello! how could i help you?

you : what is the reason for a couple of third party remote employees who is moving an oil cylinder?
When the two employees are preparing to move an oil cylinder on a mobile platform of about 200ks which is mounted on rails, the mobile platform derailed from the rail tracks, both the employees wore leather-type safety gloves and tried to lift the mobile platform and keep it on the rail tracks, while placing the mobile platform on the rail tracks, one of the employee's right hand got trapped between the rail and the mobile platform, At the incident of placing a mobile platform on the rail, the employee's right hand got trapped between the rail and the mobile platform and caused a bruised wound on the index finger of the right hand, fortunately there was no fracture, It is an accident at a level of III and its potential le